In [1]:
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
sc.stop()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MeteorologiaPySpark").getOrCreate()

import pyspark.sql.functions as F



In [2]:
ruta_csv = "temp_historico.csv"

df = spark.read.csv(ruta_csv,header=True, inferSchema=True, sep=",")

In [3]:
df.printSchema()
df.toPandas()

root
 |-- fecha: date (nullable = true)
 |-- indicativo: string (nullable = true)
 |-- provincia: string (nullable = true)
 |-- velmedia: string (nullable = true)
 |-- velmax: string (nullable = true)
 |-- tmed: string (nullable = true)
 |-- tmax: string (nullable = true)
 |-- tmin: string (nullable = true)



,fecha,indicativo,provincia,velmedia,velmax,tmed,tmax,tmin
0,2025-02-11,2918Y,SALAMANCA,"4,4","17,5","8,4","10,6","6,1"
1,2025-02-11,8293X,VALENCIA,"1,1","6,1","12,6","19,5","5,7"
2,2025-02-11,8492X,CASTELLON,"2,2","8,3","10,4","17,4","3,5"
3,2025-02-11,8270X,VALENCIA,"0,6","4,4","13,3","18,8","7,8"
4,2025-02-11,2400E,PALENCIA,"3,1","8,9","8,2","12,1","4,4"
...,...,...,...,...,...,...,...,...
1393460,2013-08-30,4193Y,CIUDAD REAL,None,None,"22,0","28,7","15,3"
1393461,2013-08-30,4478X,BADAJOZ,"2,2","7,5","25,8","35,0","16,7"
1393462,2013-08-30,7261X,ALICANTE,"4,2","10,8","22,4","26,6","18,3"
1393463,2013-08-30,B278,BALEARES,"4,2","9,7","23,0","27,2","18,9"


In [4]:
df.describe().toPandas()

,summary,indicativo,provincia,velmedia,velmax,tmed,tmax,tmin
0,count,1393465,1393465,1110442,1103708,1362777,1363316,1363098
1,mean,4421.5411627133235,None,None,None,None,None,None
2,stddev,3085.431096535091,None,None,None,None,None,None
3,min,0002I,A CORUÑA,"0,0","0,0","-0,1","-0,1","-0,1"
4,max,C939T,ZARAGOZA,"9,7","9,7","9,9","9,9","9,9"


In [5]:
df = df.orderBy('fecha')

In [6]:
df.toPandas()

,fecha,indicativo,provincia,velmedia,velmax,tmed,tmax,tmin
0,2013-08-30,0255B,BARCELONA,None,None,"22,7","30,2","15,2"
1,2013-08-30,5612B,SEVILLA,"2,2","9,7","23,1","30,0","16,2"
2,2013-08-30,2885K,ZAMORA,"3,1","9,7","18,8","27,9","9,6"
3,2013-08-30,8492X,CASTELLON,"1,9","6,7","21,2","27,1","15,4"
4,2013-08-30,2182C,SEGOVIA,"2,2","7,2","17,6","25,7","9,6"
...,...,...,...,...,...,...,...,...
1393460,2025-02-11,C665T,LAS PALMAS,"0,8","5,3","13,6","18,0","9,2"
1393461,2025-02-11,4061X,TOLEDO,"1,4","8,9",None,None,None
1393462,2025-02-11,2096B,SORIA,"3,9","10,6","7,8","11,6","4,0"
1393463,2025-02-11,2140A,SEGOVIA,"4,7","9,4","8,6","12,1","5,1"


In [7]:
for columna in df.columns:
    nulos = df.filter(F.col(columna).isNull()).count()
    print(f"Columna {columna} tiene {nulos} valores nulos")

Columna fecha tiene 0 valores nulos
Columna indicativo tiene 0 valores nulos
Columna provincia tiene 0 valores nulos
Columna velmedia tiene 283023 valores nulos
Columna velmax tiene 289757 valores nulos
Columna tmed tiene 30688 valores nulos
Columna tmax tiene 30149 valores nulos
Columna tmin tiene 30367 valores nulos


In [8]:
vars_interes = ['velmedia', 'velmax', 'tmed', 'tmax', 'tmin']

df = df.withColumn(
    "COMPLETO",
    (F.col("velmedia").isNotNull()) &
    (F.col("velmax").isNotNull()) &
    (F.col("tmed").isNotNull()) &
    (F.col("tmax").isNotNull()) &
    (F.col("tmin").isNotNull())
)

In [9]:
df_check = df.groupBy("provincia", "fecha").agg(F.max(F.col("COMPLETO").cast("int")).alias("TIENE_COMPLETO"))
df_check = df_check.withColumn("TIENE_COMPLETO", F.col("TIENE_COMPLETO").cast("boolean"))


In [10]:
df_dia_incompleto = df_check.filter(F.col("TIENE_COMPLETO") == False)

df_dia_incompleto.toPandas()

,provincia,fecha,TIENE_COMPLETO
0,CEUTA,2021-11-18,False
1,MELILLA,2024-11-19,False
2,CEUTA,2021-11-19,False
3,CEUTA,2020-11-04,False
4,MELILLA,2022-04-04,False
5,CEUTA,2021-11-21,False
6,MELILLA,2024-11-18,False
7,CEUTA,2021-11-20,False
8,MELILLA,2022-03-29,False
9,CEUTA,2016-04-22,False


In [11]:
df_2 = df.filter(~F.col("provincia").isin("CEUTA", "MELILLA"))
df_2.toPandas()

,fecha,indicativo,provincia,velmedia,velmax,tmed,tmax,tmin,COMPLETO
0,2013-08-30,0255B,BARCELONA,None,None,"22,7","30,2","15,2",False
1,2013-08-30,5612B,SEVILLA,"2,2","9,7","23,1","30,0","16,2",True
2,2013-08-30,2885K,ZAMORA,"3,1","9,7","18,8","27,9","9,6",True
3,2013-08-30,8492X,CASTELLON,"1,9","6,7","21,2","27,1","15,4",True
4,2013-08-30,2182C,SEGOVIA,"2,2","7,2","17,6","25,7","9,6",True
...,...,...,...,...,...,...,...,...,...
1390210,2025-02-11,C665T,LAS PALMAS,"0,8","5,3","13,6","18,0","9,2",True
1390211,2025-02-11,4061X,TOLEDO,"1,4","8,9",None,None,None,False
1390212,2025-02-11,2096B,SORIA,"3,9","10,6","7,8","11,6","4,0",True
1390213,2025-02-11,2140A,SEGOVIA,"4,7","9,4","8,6","12,1","5,1",True


In [12]:
for var in vars_interes:
    df_2 = df_2.withColumn(
        var,
        F.regexp_replace(F.col(var).cast("string"), ",", ".").cast("double")
    )

df_2.show(5)

+----------+----------+---------+--------+------+----+----+----+--------+
|     fecha|indicativo|provincia|velmedia|velmax|tmed|tmax|tmin|COMPLETO|
+----------+----------+---------+--------+------+----+----+----+--------+
|2013-08-30|     0255B|BARCELONA|    NULL|  NULL|22.7|30.2|15.2|   false|
|2013-08-30|     8270X| VALENCIA|     0.6|   4.7|19.7|23.6|15.8|    true|
|2013-08-30|     5612B|  SEVILLA|     2.2|   9.7|23.1|30.0|16.2|    true|
|2013-08-30|     2885K|   ZAMORA|     3.1|   9.7|18.8|27.9| 9.6|    true|
|2013-08-30|     8492X|CASTELLON|     1.9|   6.7|21.2|27.1|15.4|    true|
+----------+----------+---------+--------+------+----+----+----+--------+
only showing top 5 rows



In [13]:
df_2.show()

+----------+----------+----------+--------+------+----+----+----+--------+
|     fecha|indicativo| provincia|velmedia|velmax|tmed|tmax|tmin|COMPLETO|
+----------+----------+----------+--------+------+----+----+----+--------+
|2013-08-30|     0255B| BARCELONA|    NULL|  NULL|22.7|30.2|15.2|   false|
|2013-08-30|     9775X|    LLEIDA|    NULL|  NULL|22.5|27.5|17.5|   false|
|2013-08-30|     5612B|   SEVILLA|     2.2|   9.7|23.1|30.0|16.2|    true|
|2013-08-30|     2885K|    ZAMORA|     3.1|   9.7|18.8|27.9| 9.6|    true|
|2013-08-30|     8492X| CASTELLON|     1.9|   6.7|21.2|27.1|15.4|    true|
|2013-08-30|     2182C|   SEGOVIA|     2.2|   7.2|17.6|25.7| 9.6|    true|
|2013-08-30|     8270X|  VALENCIA|     0.6|   4.7|19.7|23.6|15.8|    true|
|2013-08-30|     4520X|   BADAJOZ|     1.4|   5.6|21.8|28.9|14.8|    true|
|2013-08-30|     2918Y| SALAMANCA|     2.2|   6.4|17.0|25.0| 9.0|    true|
|2013-08-30|     8293X|  VALENCIA|     1.1|   6.1|20.9|24.4|17.4|    true|
|2013-08-30|     2400E|  

In [14]:

for var in vars_interes:
    agg_df = df_2.groupBy("provincia", "fecha").agg(
        F.avg(var).alias(var + "_avg")
    )
    
    df_2 = df_2.join(agg_df, on=["provincia", "fecha"], how="left")
    
    df_2 = df_2.withColumn(
        var,
        F.when(F.col(var).isNull(), F.col(var + "_avg")).otherwise(F.col(var))
    )
    
    df_2 = df_2.drop(var + "_avg")

df_2.show(10)

+---------+----------+----------+--------+------+----+----+----+--------+
|provincia|     fecha|indicativo|velmedia|velmax|tmed|tmax|tmin|COMPLETO|
+---------+----------+----------+--------+------+----+----+----+--------+
| A CORUÑA|2020-03-25|      1428|     3.1|   9.2|14.0|20.2| 7.8|    true|
| ALBACETE|2020-03-25|     8178D|     0.8|   9.2| 9.0|14.4| 3.7|    true|
|  BADAJOZ|2020-03-25|     4386B|     2.5|  10.6|12.3|18.0| 6.6|    true|
|  BADAJOZ|2025-02-11|     4520X|     3.9|  12.8| 8.6|11.5| 5.7|    true|
|    CADIZ|2020-03-25|     5911A|     1.4|   4.4| 8.4|10.6| 6.1|    true|
|CASTELLON|2025-02-11|     8492X|     2.2|   8.3|10.4|17.4| 3.5|    true|
|     JAEN|2020-03-25|     5298X|     1.7|   7.2|15.9|21.5|10.3|    true|
|     LUGO|2020-03-25|     1297E|     2.2|   6.9|10.8|16.1| 5.5|    true|
|   MADRID|2020-03-25|     3110C|     2.5|   9.2| 7.0|14.7|-0.7|    true|
|   MADRID|2025-02-11|      2462|     5.3|  18.1| 1.0| 2.7|-0.6|    true|
+---------+----------+----------+-----

In [15]:
df_2.show()

+-------------+----------+----------+------------------+------------------+-----------------+-----------------+-----------------+--------+
|    provincia|     fecha|indicativo|          velmedia|            velmax|             tmed|             tmax|             tmin|COMPLETO|
+-------------+----------+----------+------------------+------------------+-----------------+-----------------+-----------------+--------+
|     A CORUÑA|2020-03-25|      1428|               3.1|               9.2|             14.0|             20.2|              7.8|    true|
|     A CORUÑA|2020-03-25|      1400|               6.1|              13.3|             11.6|             13.9|              9.3|    true|
|     A CORUÑA|2025-02-11|     1354C|3.7300000000000004|11.333333333333334|             12.9|             16.1|              9.7|   false|
|     ALBACETE|2020-03-25|     8178D|               0.8|               9.2|              9.0|             14.4|              3.7|    true|
|      ALMERIA|2020-03-25| 

In [16]:
for columna in df_2.columns:
    nulos = df_2.filter(F.col(columna).isNull()).count()
    print(f"Columna {columna} tiene {nulos} valores nulos")

Columna provincia tiene 0 valores nulos
Columna fecha tiene 0 valores nulos
Columna indicativo tiene 0 valores nulos
Columna velmedia tiene 0 valores nulos
Columna velmax tiene 0 valores nulos
Columna tmed tiene 0 valores nulos
Columna tmax tiene 0 valores nulos
Columna tmin tiene 0 valores nulos
Columna COMPLETO tiene 0 valores nulos


In [17]:
df_2 = df_2.orderBy('fecha')

In [18]:
df_2.show()

+---------+----------+----------+-----------------+-----------------+----+----+----+--------+
|provincia|     fecha|indicativo|         velmedia|           velmax|tmed|tmax|tmin|COMPLETO|
+---------+----------+----------+-----------------+-----------------+----+----+----+--------+
|   LLEIDA|2013-08-30|     9689X|2.023076923076923|8.115384615384615|15.4|21.9| 9.0|   false|
|   LLEIDA|2013-08-30|     9775X|2.023076923076923|8.115384615384615|22.5|27.5|17.5|   false|
|   LLEIDA|2013-08-30|     9772X|2.023076923076923|8.115384615384615|20.3|24.9|15.7|   false|
|   LLEIDA|2013-08-30|     9994X|              1.4|              6.9|16.1|21.9|10.3|    true|
|   LLEIDA|2013-08-30|     9771C|              1.4|              8.1|23.5|29.0|18.0|    true|
|   LLEIDA|2013-08-30|     9657X|              2.2|              8.6|16.6|24.8| 8.5|    true|
|   LLEIDA|2013-08-30|      9590|              2.2|              9.4|16.8|25.2| 8.4|    true|
|   LLEIDA|2013-08-30|     9638D|              4.7|         

In [19]:
df_2.filter((F.col("provincia") == "MALAGA") & (F.col("fecha") == "2013-08-30")).show()


+---------+----------+----------+--------+-----------------+----+----+----+--------+
|provincia|     fecha|indicativo|velmedia|           velmax|tmed|tmax|tmin|COMPLETO|
+---------+----------+----------+--------+-----------------+----+----+----+--------+
|   MALAGA|2013-08-30|     6143X|    2.45|7.659999999999999|23.8|28.3|19.3|   false|
|   MALAGA|2013-08-30|     6040X|     0.6|              6.1|23.0|28.4|17.6|    true|
|   MALAGA|2013-08-30|     6088X|     1.7|              6.1|24.3|27.0|21.6|    true|
|   MALAGA|2013-08-30|     6084X|     2.2|              5.8|25.8|28.1|23.5|    true|
|   MALAGA|2013-08-30|     6156X|     2.8|              7.2|25.8|29.3|22.2|    true|
|   MALAGA|2013-08-30|     6375X|     2.5|              8.6|23.2|30.0|16.4|    true|
|   MALAGA|2013-08-30|     6069X|     1.7|              8.9|23.0|26.7|19.2|    true|
|   MALAGA|2013-08-30|     6058I|     3.3|              7.2|25.6|27.9|23.4|    true|
|   MALAGA|2013-08-30|     6201X|     2.5|              7.5|24.8|

In [20]:
df_2 = df_2.drop("COMPLETO")


In [21]:
df_2.show(5)

+---------+----------+----------+-----------------+-----------------+----+----+----+
|provincia|     fecha|indicativo|         velmedia|           velmax|tmed|tmax|tmin|
+---------+----------+----------+-----------------+-----------------+----+----+----+
|   LLEIDA|2013-08-30|     9775X|2.023076923076923|8.115384615384615|22.5|27.5|17.5|
|   LLEIDA|2013-08-30|      9590|              2.2|              9.4|16.8|25.2| 8.4|
|   LLEIDA|2013-08-30|     9772X|2.023076923076923|8.115384615384615|20.3|24.9|15.7|
|   LLEIDA|2013-08-30|     9994X|              1.4|              6.9|16.1|21.9|10.3|
|   LLEIDA|2013-08-30|     9771C|              1.4|              8.1|23.5|29.0|18.0|
+---------+----------+----------+-----------------+-----------------+----+----+----+
only showing top 5 rows



In [22]:
df_groupby_prov_fechas = df_2.groupBy("provincia", "fecha").agg(
    F.mean("velmedia").alias("velmedia"),
    F.mean("velmax").alias("velmax"),
    F.mean("tmed").alias("tmed"),
    F.mean("tmax").alias("tmax"),
    F.mean("tmin").alias("tmin")
)

df_groupby_prov_fechas.show()

+----------+----------+------------------+------------------+------------------+------------------+------------------+
| provincia|     fecha|          velmedia|            velmax|              tmed|              tmax|              tmin|
+----------+----------+------------------+------------------+------------------+------------------+------------------+
|    LLEIDA|2013-08-30| 2.023076923076923| 8.115384615384617| 18.67142857142857|25.009523809523806|12.352380952380955|
|    BURGOS|2013-08-30|3.3416666666666663| 9.533333333333333|16.415384615384617|              24.0| 8.815384615384616|
|VALLADOLID|2013-08-30|            1.9125| 8.924999999999999|18.729999999999997|27.330000000000002|             10.12|
|   SEGOVIA|2013-08-30|2.0142857142857142| 8.842857142857143|            16.975|25.112499999999997|             8.875|
|  ALICANTE|2013-08-30|2.4999999999999996| 9.825000000000001| 21.19090909090909|25.181818181818183|17.227272727272727|
|    MALAGA|2013-08-30|              2.45|      

In [23]:
df_groupby_prov_fechas.toPandas()

,provincia,fecha,velmedia,velmax,tmed,tmax,tmin
0,LLEIDA,2013-08-30,2.023077,8.115385,18.671429,25.009524,12.352381
1,BURGOS,2013-08-30,3.341667,9.533333,16.415385,24.000000,8.815385
2,VALLADOLID,2013-08-30,1.912500,8.925000,18.730000,27.330000,10.120000
3,SEGOVIA,2013-08-30,2.014286,8.842857,16.975000,25.112500,8.875000
4,ALICANTE,2013-08-30,2.500000,9.825000,21.190909,25.181818,17.227273
...,...,...,...,...,...,...,...
84027,VALLADOLID,2025-02-11,2.750000,8.812500,8.811111,14.233333,3.411111
84028,MURCIA,2025-02-11,1.809524,6.842857,12.462963,18.017857,6.840741
84029,HUELVA,2025-02-11,4.445455,13.418182,11.985714,15.514286,8.450000
84030,GUADALAJARA,2025-02-11,2.250000,9.350000,7.611111,12.144444,3.077778


In [24]:
df_2.dtypes

[('provincia', 'string'),
 ('fecha', 'date'),
 ('indicativo', 'string'),
 ('velmedia', 'double'),
 ('velmax', 'double'),
 ('tmed', 'double'),
 ('tmax', 'double'),
 ('tmin', 'double')]

In [25]:
import math

In [26]:
df_groupby_prov_fechas = df_groupby_prov_fechas.withColumn("day_of_year", F.dayofyear("fecha"))

df_groupby_prov_fechas = df_groupby_prov_fechas.withColumn("sin_day_of_year",F.sin(F.col("day_of_year") * F.lit(2 * math.pi / 366)))
df_groupby_prov_fechas = df_groupby_prov_fechas.withColumn("cos_day_of_year",F.cos(F.col("day_of_year") * F.lit(2 * math.pi / 366)))

In [27]:
df_groupby_prov_fechas.toPandas()

,provincia,fecha,velmedia,velmax,tmed,tmax,tmin,day_of_year,sin_day_of_year,cos_day_of_year
0,LLEIDA,2013-08-30,2.023077,8.115385,18.671429,25.009524,12.352381,242,-0.848351,-0.529434
1,BURGOS,2013-08-30,3.341667,9.533333,16.415385,24.000000,8.815385,242,-0.848351,-0.529434
2,VALLADOLID,2013-08-30,1.912500,8.925000,18.730000,27.330000,10.120000,242,-0.848351,-0.529434
3,SEGOVIA,2013-08-30,2.014286,8.842857,16.975000,25.112500,8.875000,242,-0.848351,-0.529434
4,ALICANTE,2013-08-30,2.500000,9.825000,21.190909,25.181818,17.227273,242,-0.848351,-0.529434
...,...,...,...,...,...,...,...,...,...,...
84027,VALLADOLID,2025-02-11,2.750000,8.812500,8.811111,14.233333,3.411111,42,0.660152,0.751132
84028,MURCIA,2025-02-11,1.809524,6.842857,12.462963,18.017857,6.840741,42,0.660152,0.751132
84029,HUELVA,2025-02-11,4.445455,13.418182,11.985714,15.514286,8.450000,42,0.660152,0.751132
84030,GUADALAJARA,2025-02-11,2.250000,9.350000,7.611111,12.144444,3.077778,42,0.660152,0.751132


In [28]:
df_groupby_prov_fechas = df_groupby_prov_fechas.drop("fecha", "day_of_year")


In [29]:
df_groupby_prov_fechas.toPandas()

,provincia,velmedia,velmax,tmed,tmax,tmin,sin_day_of_year,cos_day_of_year
0,LLEIDA,2.023077,8.115385,18.671429,25.009524,12.352381,-0.848351,-0.529434
1,BURGOS,3.341667,9.533333,16.415385,24.000000,8.815385,-0.848351,-0.529434
2,VALLADOLID,1.912500,8.925000,18.730000,27.330000,10.120000,-0.848351,-0.529434
3,SEGOVIA,2.014286,8.842857,16.975000,25.112500,8.875000,-0.848351,-0.529434
4,ALICANTE,2.500000,9.825000,21.190909,25.181818,17.227273,-0.848351,-0.529434
...,...,...,...,...,...,...,...,...
84027,VALLADOLID,2.750000,8.812500,8.811111,14.233333,3.411111,0.660152,0.751132
84028,MURCIA,1.809524,6.842857,12.462963,18.017857,6.840741,0.660152,0.751132
84029,HUELVA,4.445455,13.418182,11.985714,15.514286,8.450000,0.660152,0.751132
84030,GUADALAJARA,2.250000,9.350000,7.611111,12.144444,3.077778,0.660152,0.751132


In [30]:
for columna in df_groupby_prov_fechas.columns:
    nulos = df_groupby_prov_fechas.filter(F.col(columna).isNull()).count()
    print(f"Columna {columna} tiene {nulos} valores nulos")

Columna provincia tiene 0 valores nulos
Columna velmedia tiene 0 valores nulos
Columna velmax tiene 0 valores nulos
Columna tmed tiene 0 valores nulos
Columna tmax tiene 0 valores nulos
Columna tmin tiene 0 valores nulos
Columna sin_day_of_year tiene 0 valores nulos
Columna cos_day_of_year tiene 0 valores nulos


In [31]:
# Guardar en CSV (esto crea una carpeta con múltiples archivos CSV)
df_groupby_prov_fechas.write.option("header", "true").mode("overwrite").csv("datos_limpios.csv")

# Si deseas un único archivo CSV, puedes reducir a una partición:
df_groupby_prov_fechas.coalesce(1).write.option("header", "true").mode("overwrite").csv("datos_limpios_2")

# Guardar en Parquet (esto crea un directorio con archivos Parquet)
df_groupby_prov_fechas.write.mode("overwrite").parquet("datos_limpios_3_prq")